In [53]:
!pip install transformers datasets torch sentencepiece accelerate

In [54]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW

In [55]:
df = pd.read_csv("/content/data.csv")

In [56]:
df[["happy", "sad", "anger", "fear"]] /= 100

In [57]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [58]:
def tokenize_data(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

In [59]:
df["tokens"] = df["Paragraph"].apply(lambda x: tokenize_data(x))

In [60]:
class TeluguEmotionDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.labels = torch.tensor(self.data[["happy", "sad", "anger", "fear"]].values, dtype=torch.float)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokens = self.data.iloc[idx]["tokens"]
        return {
            "input_ids": tokens["input_ids"].squeeze(),
            "attention_mask": tokens["attention_mask"].squeeze(),
            "labels": self.labels[idx]
        }

In [61]:
dataset = TeluguEmotionDataset(df)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [62]:
class EnhancedEmotionClassifier(nn.Module):
    def __init__(self, num_labels=4):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.dropout = nn.Dropout(0.3)
        self.dense = nn.Linear(self.bert.config.hidden_size, 256)
        self.classifier = nn.Linear(256, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        features = F.relu(self.dense(pooled_output))
        logits = self.classifier(features)
        return logits

In [63]:
model = EnhancedEmotionClassifier().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
# loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.BCEWithLogitsLoss()
# Or weighted loss for class imbalance:
# weights = torch.tensor([2.0, 1.0, 1.0, 1.0])  # Adjust based on your data
# loss_fn = nn.CrossEntropyLoss(weight=weights.to(device))

In [64]:
epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

print("Training complete!")

Epoch 1, Loss: 0.5316507470878687
Epoch 2, Loss: 0.503074321557175
Epoch 3, Loss: 0.4991218386725946
Epoch 4, Loss: 0.49811653657393024
Epoch 5, Loss: 0.49844642118974164
Epoch 6, Loss: 0.490732725371014
Epoch 7, Loss: 0.48683805560523813
Epoch 8, Loss: 0.4860095571387898
Epoch 9, Loss: 0.47723981873555615
Epoch 10, Loss: 0.47106841477480804
Epoch 11, Loss: 0.4669351652264595
Epoch 12, Loss: 0.4644781445915049
Epoch 13, Loss: 0.4617360965772109
Epoch 14, Loss: 0.46018194204027
Epoch 15, Loss: 0.45936497233130713
Epoch 16, Loss: 0.4582851766185327
Epoch 17, Loss: 0.4584353945472024
Epoch 18, Loss: 0.45719874785705045
Epoch 19, Loss: 0.4570943164554509
Epoch 20, Loss: 0.4567573903636499
Epoch 21, Loss: 0.45669353821060876
Epoch 22, Loss: 0.4559469223022461
Epoch 23, Loss: 0.4555663039738482
Epoch 24, Loss: 0.45583452690731396
Epoch 25, Loss: 0.4553373130885037
Epoch 26, Loss: 0.45534400709650735
Epoch 27, Loss: 0.45531690459359775
Epoch 28, Loss: 0.4552532149986787
Epoch 29, Loss: 0.4550

In [68]:
def predict_emotions(model, tokenizer, sentence):
    model.eval()
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    with torch.no_grad():
        outputs = model(inputs["input_ids"], inputs["attention_mask"])

    probabilities = F.softmax(outputs, dim=1).cpu().numpy().flatten()
    emotions = ["Happy", "Sad", "Anger", "Fear"]
    return {emotions[i]: f"{probabilities[i]*100:.2f}%" for i in range(len(emotions))}

new_sentence = "అయినా ముందుకు అడుగు వేయలేకపోయాడు బాటసారి.‘‘ముసలిదాన్ని. లేవలేను. పరిగెత్తలేను. చేతి గోళ్ళూ, కాలి గోళ్ళూ రెండూ మొద్దుబారిపోయాయి. పళ్ళూడిపోయాయి. కళ్ళు కూడా సరిగా కనిపించడం లేదు. మాంసం అన్న ముచ్చటే లేదు. మాంసం తినడాన్ని ఎప్పుడో మానుకున్నాను. పండ్లూ కాయలూ తిని బతుకుతున్నాను. నా అవతారం చూసావు కదా! నమ్మకం కలగడం లేదా? నా గురించి లేనిపోనివి ఆలోచించి భయపడకు. రా! ముందు ఆ చెరువులో స్నానం చెయ్‌.’’ అన్నది పులి.దాని మాటలు నమ్మాడు బాటసారి. పులి మాటలు నమ్మాడనడం కన్నా, బంగారం మీద ఆశ ఎక్కువయిందతనికి. బంగారు కడియం కళ్ళ ముందు కదలాడుతోంటే స్నానం చేసేందుకు చెరువులోనికి దిగాడు. బురద బురదగా ఉంది లోపల. కాలు పెడితే జారి పోతున్నాడు"
predicted_emotions = predict_emotions(model, tokenizer, new_sentence)

print("Predicted Emotion Percentages:")
for emotion, percentage in predicted_emotions.items():
    print(f"{emotion}: {percentage}")


Predicted Emotion Percentages:
Happy: 47.01%
Sad: 29.12%
Anger: 7.10%
Fear: 16.78%


In [69]:
import torch
import torch.nn.functional as F

def predict_emotions(model, tokenizer, sentence):
    model.eval()
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    with torch.no_grad():
        outputs = model(inputs["input_ids"], inputs["attention_mask"])

    probabilities = F.softmax(outputs, dim=1).cpu().numpy().flatten()
    emotions = ["Happy", "Sad", "Anger", "Fear"]
    return {emotions[i]: f"{probabilities[i]*100:.2f}%" for i in range(len(emotions))}

new_sentence = "పరిష్కారమార్గాన్ని కనుక్కోవాలనుకున్నాడు. మంచి గురువుల దగ్గర చదివిస్తే పిల్లలెందుకు చదవరు? చదవనని వారనలేదే! చదివించడం లేదు కాబట్టే వారాడుతున్నారు. తప్పు తనదే! గారాబం చేసి, అప్పుడే పిల్లలకు చదువులెందుకనుకుంటూ నిర్లక్ష్యం చేసి, ఇప్పుడు బాధపడడంలో అర్థం లేదనుకున్నాడు. పిల్లలకు చదువు చెప్పించడం తల్లిదండ్రుల బాధ్యత. సరయిన గురువు దగ్గర వారిని చదివించాలి. చదివించకపోవడం నేరం కూడా అనుకున్నాడు. పుట్టుకతోనే ఎవరూ పండితులు కారు. విద్వాంసులు అంతకన్నా కారు. బాగా చదువుకుని పండితులవుతారు. తర్వాత్తర్వాత విద్వాంసులుగా కీర్తి ప్రతిష్ఠలు ఆర్జిస్తారు. ప్రయత్నించాలి. ప్రయత్నిస్తేనే ఏదయినా ఫలిస్తుందనుకున్నాడు. వూరికే దిగులుపడి ప్రయోజనం లేదు. "


model = EnhancedEmotionClassifier()
model.load_state_dict(torch.load("/content/enhanced_emotion_classifier.pth"))
model.to(device)
model.eval()

predicted_emotions = predict_emotions(model, tokenizer, new_sentence)

print("Predicted Emotion Percentages:")
for emotion, percentage in predicted_emotions.items():
    print(f"{emotion}: {percentage}")

Predicted Emotion Percentages:
Happy: 48.45%
Sad: 28.11%
Anger: 7.04%
Fear: 16.39%
